In [1]:
base_dir = "D:/knowknow/infosci _ dataset"

In [2]:
#name_blacklist = [    "*us", 'Press', 'Knopf', '(January', 'Co', 'London', 'Bros', 'Books', 'Wilson','[anonymous]']
name_blacklist = []

In [1]:
from knowknow import *
from knowknow.datasources import wos

In [3]:
prelim_count = Dataset("infosci-wos-all")

loading variable infosci-wos-all/_attributes from disk
loading variable infosci-wos-all/groups from disk


In [4]:
filtered = Dataset("infosci-wos-filtered")
filtered.groupings = [] # clear... not yet

loading variable infosci-wos-filtered/_attributes from disk
loading variable infosci-wos-filtered/groups from disk
loading variable infosci-wos-filtered/group_reps from disk


In [5]:
final = Dataset("infosci-wos-filtered-grouped")

loading variable infosci-wos-filtered-grouped/_attributes from disk
loading variable infosci-wos-filtered-grouped/groups from disk
loading variable infosci-wos-filtered-grouped/group_reps from disk


In [6]:
final.set_attribute('doi', '10.7910/DVN/RHJ8UX')

# Counting over the entire dataset

In [7]:
#if not prelim_count.exists():
cnt = wos.counter(
    wos_txt_base = base_dir,
    output_database = prelim_count.name,
    RUN_EVERYTHING = False,
    name_blacklist=name_blacklist
)

cnt.run()

File 0/678: savedrecs.txt
Document: 0
Citations: 0
File 50/678: savedrecs-18.txt
Document: 11463
Citations: 227625
File 100/678: savedrecs-2.txt
Document: 25383
Citations: 531187
File 150/678: savedrecs-8.txt
Document: 38842
Citations: 757912
File 200/678: 201-500.txt
Document: 50043
Citations: 912316
File 250/678: savedrecs.txt
Document: 65195
Citations: 1005083
File 300/678: 1501-2000savedrecs.txt
Document: 80596
Citations: 1199339
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._86 (1).txt. Skipping.
Cannot load file! ._86 (2).txt. Skipping.
Cannot load file! ._86 (3).txt. Skipping.
Cannot load file! ._1-500.txt. Skipping.
Cannot load file! ._1001-1500.txt. Skipping.
Cannot load file! ._1501-2000.txt. Skipping.
Cannot load file! ._2001-2500.txt. Skipping.
Cannot load file! ._2501-3000.txt. Skipping.
Cannot load file! ._3001-3500.txt. Skipping.
Cannot load file! ._3501-4000.txt. Skipping.
Cannot load file! ._4001-4500.txt. 

In [8]:
cnt.save_counters()

loading variable infosci-wos-all/groups from disk


In [9]:
prelim_count.save_variable('_attributes', {})

In [10]:
prelim_count.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
prelim_count.set_attribute('RELIABLE_DATA_ENDS_HERE', 2020)
prelim_count.set_attribute('type','wos')

In [11]:
max( prelim_count.items('fy') )

loading variable infosci-wos-all/ind ___ fy from disk


2022

# Examine journals to determine what to drop

In [12]:
zero_sofars = []
reasons = defaultdict(int)
# how I decide whether to keep a journal
def filt(j):
    global reasons
    
    try:
        tt = prelim_count.trend('fj', j)
    except invalid_entry:
        return False

    # get the years for which there is a positive count of BIBLIOGRAPHIES
    ys = set(y for y,c in tt._c.items() if c > 0)

    # it must have been around for at least 10 years, with their first year being sometime before 1995
    if max(ys) < 2020:
        reasons['ended before 2020'] += 1
        return False
    
    #if not max(ys)-min(ys) >= 10:
    #    reasons['not a decade'] += 1
    #    return False
    #if not min(ys) <= 1995:
    #    reasons['started after 1995'] += 1
    #    return False

    # 5 years consecutively, then drop
    ever_nonzero = False
    zero_sofar = 0
    zero_sofar_M = 0
    for y in range(tt.data_start, tt.data_end):
        c = tt._c[ y ]
        
        if c > 0:
            ever_nonzero = True

        if not ever_nonzero:
            continue

        if c == 0:
            zero_sofar += 1
        else:
            zero_sofar_M = max(zero_sofar_M, zero_sofar)
            zero_sofar = 0

        if zero_sofar >= 3:
            reasons['3 consecutive with zero'] += 1
            return False
        
    # at least 100 papers published total
    if not prelim_count(fj=j).docs >= 100:
        reasons['needs 100 papers'] += 1
        return False
    
    zero_sofars.append(zero_sofar_M)
    #if zero_sofar_M > 0:
    #    print(j,zero_sofar_M)

    return True

In [13]:
journals = list(prelim_count.items('fj'))

print( "There are %s journals total in `%s`" % (len(journals), prelim_count.name) )

keep, drop = set(), set()
for j in journals:
    if filt(j):
        keep.add(j)
    else:
        drop.add(j)
        
print( "Only %s meet my standards" % (len(keep) ) )

loading variable infosci-wos-all/ind ___ fj from disk
There are 88 journals total in `infosci-wos-all`
loading variable infosci-wos-all/ind ___ fj.fy from disk
loading variable infosci-wos-all/doc ___ fj from disk
Only 77 meet my standards


In [14]:
reasons

defaultdict(int, {'3 consecutive with zero': 5, 'ended before 2020': 6})

In [15]:
Counter(zero_sofars)

Counter({0: 76, 1: 1})

## journals to keep

In [16]:
print(", ".join(sorted(keep)))

african journal of library archives and information science, aslib journal of information management, canadian journal of information and library science-revue canadienne des sciences de l information et de bibliotheconomie, college & research libraries, data technologies and applications, electronic library, european journal of information systems, government information quarterly, health information and libraries journal, informacao & sociedade-estudos, informacios tarsadalom, information & culture, information & management, information and organization, information development, information processing & management, information research-an international electronic journal, information society, information systems journal, information systems research, information technology & management, information technology & people, information technology and libraries, information technology for development, international journal of computer-supported collaborative learning, international journal

## journals to drop

In [17]:
print(", ".join(sorted(drop)))

data base for advances in information systems, econtent, ethics and information technology, information management, journal of the american society for information science and technology, library hi tech, library journal, libri-international journal of libraries and information studies, reference & user services quarterly, social science information studies, social science information sur les sciences sociales


# compute counts over just these selected journals

In [19]:
# clear the groups, this shouldn't happen yet.
cnt = wos.counter(
    wos_txt_base = base_dir,
    output_database = filtered.name,
    RUN_EVERYTHING = False,
    name_blacklist=name_blacklist,
    journals_filter=keep
)

cnt.run()

File 0/678: savedrecs.txt
Document: 0
Citations: 0
File 50/678: savedrecs-18.txt
Document: 10035
Citations: 210107
File 100/678: savedrecs-2.txt
Document: 23883
Citations: 516906
File 150/678: savedrecs-8.txt
Document: 37342
Citations: 744702
File 200/678: 201-500.txt
Document: 46614
Citations: 877386
File 250/678: savedrecs.txt
Document: 53786
Citations: 954795
File 300/678: 1501-2000savedrecs.txt
Document: 68269
Citations: 1141213
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._86 (1).txt. Skipping.
Cannot load file! ._86 (2).txt. Skipping.
Cannot load file! ._86 (3).txt. Skipping.
Cannot load file! ._1-500.txt. Skipping.
Cannot load file! ._1001-1500.txt. Skipping.
Cannot load file! ._1501-2000.txt. Skipping.
Cannot load file! ._2001-2500.txt. Skipping.
Cannot load file! ._2501-3000.txt. Skipping.
Cannot load file! ._3001-3500.txt. Skipping.
Cannot load file! ._3501-4000.txt. Skipping.
Cannot load file! ._4001-4500.txt. S

In [20]:
filtered.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
filtered.set_attribute('RELIABLE_DATA_ENDS_HERE', 2020)
filtered.set_attribute('type','wos')

# now compute summary statistics about citations.

There are many we will want to exclude for almost all analyses, in order to keep more cooccurrence counts

In [21]:
t = ttDF(
    dataset_name = filtered.name,
    dtype = 'c',
    debug=False
).run()

loading variable infosci-wos-filtered/groups from disk
loading variable infosci-wos-filtered/ind ___ c from disk
Item 0/1262853 (&, P.|solicitations 0707)
Current memory usage is 243.786313MB; Peak was 630.179709MB
defaultdict(<class 'int'>, {})
loading variable infosci-wos-filtered/doc ___ c from disk
loading variable infosci-wos-filtered/ind ___ c.fy from disk
loading variable infosci-wos-filtered/ind ___ fy from disk
Item 50000/1262853 (Ashby, S.|2007|ann ny acad sci,v1114,p362)
Current memory usage is 957.479375MB; Peak was 1151.258978MB
defaultdict(<class 'int'>, {'at least one citation': 50000, 'less than 5 citations... dropped.2': 46863, 'passed tests pre-blacklist': 2872, 'not enough parts': 27, 'first citation before pub date': 6, 'less than 5 citations... dropped.': 231, 'invalid entry': 1})
Item 100000/1262853 (Bergen, M.|advances in artificial intelligence. 14th biennial conference of the canadian society for computational studies of intelligence)
Current memory usage is 97

In [22]:
len( filtered.load_variable('c.ysum') )

loading variable infosci-wos-filtered/c.ysum from disk


74508

# and group those which make the cut

In [23]:
# note that unfortunately this currently requires YSUM
#   and thus will be based on the filters embedded in that...
#   AND these decisions will propagate further, because counters only consider if they're in the groups
# some decisions need to be made here...

from grouper import Grouper
g = Grouper(dataset=filtered)
g.run()

loading variable infosci-wos-filtered/c.ysum from disk
74508 strings total...
sample articles: ['2-C,, D.|2001|', '2-M,, D.|1999|', '2-M,, D.|2000|', 'Aabo, S.|2002|j libr inf sci,v34,p5', 'Aabo, S.|2005|inf econ policy,v17,p175', 'Aabo, S.|2005|new lib world,v106,p487', 'Aabo, S.|2009|new lib world,v110,p311', 'Aabo, S.|2010|libr inform sci res,v32,p16', 'Aabo, S.|2012|libr inform sci res,v34,p138', 'Aabo, V.|2005|j libr inf sci,v37,p205']
sample books: ['(Itu), I.|meas inf soc 2012', '4Milbrath, L.|political participat', '8Rs, R.|fut hum res can lib', 'Aabo, S.|value public lib met', 'Aaker, D.|brand leadership', 'Aaker, D.|building strong bran', 'Aaker, D.|managing brand equit', 'Aaker, D.|marketing res', 'Aarseth, E.|cybertext perspectiv', 'Abadal, E.|prensa digital bibli']
60651 articles, 13857 books to group
2213 books have some connection to others in a group
12615 groups total
[(9680, 15), (11568, 11), (8421, 8), (9253, 8), (5345, 7), (6511, 7), (8078, 7), (12457, 7), (2068, 6)

# recount on this focused group, grouping citations

In [24]:
cnt = wos.counter(
    wos_txt_base = base_dir,
    output_database = final.name,
    RUN_EVERYTHING = True,
    name_blacklist=name_blacklist,
    journals_filter= keep,
    groups= filtered.load_variable('groups'),
    group_reps= filtered.load_variable('group_reps'),
    #debug=True
)

loading variable infosci-wos-filtered/groups from disk
loading variable infosci-wos-filtered/group_reps from disk


In [25]:
cnt.run()

File 0/678: savedrecs.txt
Document: 0
Citations: 0
File 50/678: savedrecs-18.txt
Document: 10035
Citations: 40352
File 100/678: savedrecs-2.txt
Document: 23883
Citations: 59602
File 150/678: savedrecs-8.txt
Document: 37342
Citations: 67733
File 200/678: 201-500.txt
Document: 46614
Citations: 70040
File 250/678: savedrecs.txt
Document: 53786
Citations: 70363
File 300/678: 1501-2000savedrecs.txt
Document: 68269
Citations: 72122
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._savedrecs.txt. Skipping.
Cannot load file! ._86 (1).txt. Skipping.
Cannot load file! ._86 (2).txt. Skipping.
Cannot load file! ._86 (3).txt. Skipping.
Cannot load file! ._1-500.txt. Skipping.
Cannot load file! ._1001-1500.txt. Skipping.
Cannot load file! ._1501-2000.txt. Skipping.
Cannot load file! ._2001-2500.txt. Skipping.
Cannot load file! ._2501-3000.txt. Skipping.
Cannot load file! ._3001-3500.txt. Skipping.
Cannot load file! ._3501-4000.txt. Skipping.
Cannot load file! ._4001-4500.txt. Skipping

In [26]:
final.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
final.set_attribute('RELIABLE_DATA_ENDS_HERE', 2020)
final.set_attribute('type','wos')

# do the simple string grouper, finally...

In [27]:
from grouper import SimpleStringGrouper as grp
grp(final).run()

loading variable infosci-wos-filtered-grouped/ind ___ c from disk


# upload the counts

In [29]:
env.GLOBS

{'kk_code_dir': 'C:\\Users\\am2873\\GitHub',
 'kk_data_dir': 'D:\\knowknow\\data',
 'dataverse_api_key': '291abf80-633f-45d8-bb8e-9d30d93bda95'}

In [31]:
final.upload?

Signature:
final.upload(
    api_key,
    dataset_description='',
    dataverse_name='kk-citation-counts',
    dataverse_server='https://dataverse.harvard.edu',
)
Docstring: <no docstring>
File:      c:\users\amcga\github\knowknow\knowknow\datastore_cnts\count_cache.py
Type:      method


In [33]:
final.upload(
    env.GLOBS['dataverse_api_key'],
    dataset_description="""
        Replication of the filtering and grouping machinery 
        described in [McGail 2021](https://link.springer.com/article/10.1007/s12108-021-09490-4).
        Dataset includes 77 library and information science journals, from Web of Science.
    """,
    dataverse_name='kk-citation-counts',
    dataverse_server='https://dataverse.harvard.edu',
)

Created dataset with persistent ID = doi:10.7910/DVN/RHJ8UX
uploading doc ___ fa ...
uploading doc ___ fa.fj ...
uploading doc ___ fa.fy ...
uploading doc ___ ffa ...
uploading doc ___ ffa.fj ...
uploading doc ___ ffa.fy ...
uploading doc ___ fj ...
uploading doc ___ fj.fy ...
uploading doc ___ fj.ta ...
uploading doc ___ fj.ty ...
uploading doc ___ fy ...
uploading doc ___ fy.ta ...
uploading doc ___ fy.ty ...
uploading doc ___ ta ...
uploading doc ___ ty ...
uploading groups ...
uploading group_reps ...
uploading ind ___ c ...
uploading ind ___ c.fa ...
uploading ind ___ c.ffa ...
uploading ind ___ c.fj ...
uploading ind ___ c.fj.fy ...
uploading ind ___ c.fy ...
uploading ind ___ fa ...
uploading ind ___ fa.fj ...
uploading ind ___ fa.fy ...
uploading ind ___ ffa ...
uploading ind ___ ffa.fj ...
uploading ind ___ ffa.fy ...
uploading ind ___ fj ...
uploading ind ___ fj.fy ...
uploading ind ___ fj.ta ...
uploading ind ___ fj.ty ...
uploading ind ___ fy ...
uploading ind ___ fy.ta ...

In [2]:
dataset_metadata

{'infosci-wos-all': {'doi': 'infosci-wos-all',
  'RELIABLE_DATA_STARTS_HERE': 1900,
  'RELIABLE_DATA_ENDS_HERE': 2020,
  'type': 'wos',
  'name': 'infosci-wos-all'},
 'infosci-wos-filtered': {'doi': 'infosci-wos-filtered',
  'RELIABLE_DATA_STARTS_HERE': 1900,
  'RELIABLE_DATA_ENDS_HERE': 2020,
  'type': 'wos',
  'name': 'infosci-wos-filtered'},
 'infosci-wos-filtered-grouped': {'doi': 'infosci-wos-filtered-grouped',
  'RELIABLE_DATA_STARTS_HERE': 1950,
  'RELIABLE_DATA_ENDS_HERE': 2020,
  'type': 'wos',
  'name': 'infosci-wos-filtered-grouped'},
 'jstor-test': {'RELIABLE_DATA_STARTS_HERE': 1900,
  'RELIABLE_DATA_ENDS_HERE': 2015,
  'name': 'jstor-test'},
 'sociology-wos-74a': {'RELIABLE_DATA_STARTS_HERE': 1900,
  'RELIABLE_DATA_ENDS_HERE': 2015,
  'type': 'wos',
  'name': 'sociology-wos-74a'},
 'sociology-wos-74b': {'RELIABLE_DATA_STARTS_HERE': 1920,
  'RELIABLE_DATA_ENDS_HERE': 2015,
  'type': 'wos',
  'name': 'sociology-wos-74b'},
 'sociology-wos-74b-noself': {'RELIABLE_DATA_STARTS_H